 **Analyse des calques DWG (convertis en DXF) utilisés pour les DT/DICT** (py1) COLAB

 Ce programme permet d’obtenir une liste de l’ensemble des calques d’un dessin DWG préalablement converti en DXF.
Il va parcourir l’ensemble des entités de chaque calque et créer un fichier Excel avec :

- un champ qui répertorie l’ensemble des calques (leurs noms),

- un champ qui décrit les types de géométrie que l’on peut retrouver dans chaque calque ainsi que le nombre d’entités,

- un champ avec la couleur Hex des calques (conversion des couleurs AutoCAD),

- leur style de ligne et leur épaisseur.

Le document final permet une analyse approfondie des styles de couches, aspect important pour la charte graphique, ainsi que des types de géométrie, ce qui permettra d’élaborer une méthodologie pour extraire et transformer ces données DAO en SIG.

![image_py1](../images_colab/py1.png)

In [ ]:
!pip install ezdxf xlsxwriter
from collections import defaultdict
import ezdxf
import xlsxwriter
import ezdxf.colors

# === FONCTION DE CONVERSION COULEUR AUTOACAD → CODE COULEUR HEX ===
def convert_to_hex(color_index):
    if 1 <= color_index <= 255:
        rgb = ezdxf.colors.DXF_DEFAULT_COLORS[color_index]
        r, g, b = (rgb >> 16) & 0xFF, (rgb >> 8) & 0xFF, rgb & 0xFF
        return f"#{r:02X}{g:02X}{b:02X}"
    return "Inconnue"

# === CONFIGURATION ===
dxf_path = "/content/23-4199_Campus Université_Strasbourg_Investigation_Complementaire_ADRE_Reseaux.dxf"
excel_output = "symbologie_AUTOCAD.xlsx"

# Ouverture du fichier DXF
try:
    doc = ezdxf.readfile(dxf_path)
except IOError:
    print(f"Erreur : Impossible de lire {dxf_path}.")
    exit()

# === EXTRACTION DES CALQUES & DÉTECTION DES TYPES GÉOMÉTRIQUES ===
calques_infos = defaultdict(lambda: {"types": set(), "count": 0})

# Liste des types pris en charge par AutoCAD
dxftype_to_label = {
    "POINT": "Point",
    "LINE": "Ligne",
    "LWPOLYLINE": "Ligne",
    "POLYLINE": "Ligne",
    "INSERT": "Symbole",
    "CIRCLE": "Courbe",
    "ARC": "Courbe",
    "ELLIPSE": "Surface complexe",
    "SPLINE": "Surface complexe",
    "TEXT": "Texte",
    "MTEXT": "Texte",
    "HATCH": "Hachure",
    "DIMENSION": "Cotations",
    "LEADER": "Annotations",
    "MLEADER": "Annotations",
    "XLINE": "Guides",
    "RAY": "Guides",
    "IMAGE": "Image",
    "WIPEOUT": "Masque",
    "SHAPE": "Forme personnalisée",
    "3DFACE": "Surface 3D",
    "3DSOLID": "Solide 3D",
    "REGION": "Région",
    "BODY": "Corps 3D",
    "VIEWPORT": "Fenêtre graphique",
    "UNDERLAY": "Sous-couche",
    "GEOPOSITIONMARKER": "Repère GPS",
    "MLINE": "Ligne multiple",
    "HELIX": "Hélice",
    "SURFACE": "Surface 3D",
    "MESH": "Maillage",
}

# === PARCOURS DES ENTITÉS ===
for entity in doc.modelspace():
    layer_name = entity.dxf.layer.strip()
    dxftype = entity.dxftype()
    type_geom = dxftype_to_label.get(dxftype, f"Autre ({dxftype})")

    calques_infos[layer_name]["types"].add(type_geom)
    calques_infos[layer_name]["count"] += 1

# === AJOUT DES CALQUES VIDES ===
for layer in doc.layers:
    layer_name = layer.dxf.name.strip()
    if layer_name not in calques_infos:
        calques_infos[layer_name]["types"].add("Aucun objet")
        calques_infos[layer_name]["count"] = 0

# === FORMATAGE DES DONNÉES POUR EXCEL ===
symbologie = []
for layer_name, info in calques_infos.items():
    layer_obj = doc.layers.get(layer_name)
    type_geom_final = " & ".join(sorted(info["types"]))
    color_hex = convert_to_hex(layer_obj.dxf.color)
    line_style = layer_obj.dxf.linetype if hasattr(layer_obj.dxf, "linetype") else "Standard"
    line_weight = f"{layer_obj.dxf.lineweight / 100:.2f} mm" if layer_obj.dxf.lineweight != -3 else "Valeur par défaut"

    symbologie.append({
        "Calque": layer_name,
        "Type géométrique": type_geom_final,
        "Nombre d’objets": info["count"],
        "Couleur (Hex)": color_hex,
        "Style de ligne": line_style,
        "Épaisseur": line_weight,
    })

# === EXPORT EXCEL ===
workbook = xlsxwriter.Workbook(excel_output)
worksheet = workbook.add_worksheet("Symbologie AutoCAD")

# Titres des colonnes
titles = ["Calque", "Type géométrique", "Nombre d’objets", "Couleur (Hex)", "Style de ligne", "Épaisseur"]
bold_format = workbook.add_format({"bold": True})
border_format = workbook.add_format({"border": 1})

for col, title in enumerate(titles):
    worksheet.write(0, col, title, bold_format)

# Données
for row, data in enumerate(symbologie, start=1):
    worksheet.write(row, 0, data["Calque"], border_format)
    worksheet.write(row, 1, data["Type géométrique"], border_format)
    worksheet.write(row, 2, data["Nombre d’objets"], border_format)

    color_format = workbook.add_format({
        "bg_color": "#CCCCCC" if data["Couleur (Hex)"] == "Inconnue" else data["Couleur (Hex)"],
        "border": 1
    })
    worksheet.write(row, 3, data["Couleur (Hex)"], color_format)

    worksheet.write(row, 4, data["Style de ligne"], border_format)
    worksheet.write(row, 5, data["Épaisseur"], border_format)

# Ajustement automatique
worksheet.set_column("A:F", 22)

# Sauvegarde
workbook.close()


**Séparation automatique des tronçons (linéaires) des réseaux dans des shapefiles séparés selon leur type** (py2)Arcgis pro

Ce programme permet d’obtenir des shapefiles séparés pour l’ensemble des types de réseaux à partir de la couche SIG via le ModelBuilder d’ArcGIS Pro.

À partir d’une liste générée automatiquement grâce au champ « type », il va ensuite créer des couches où il séparera individuellement chaque entité.

![image_py2](../images_colab/py2.png)

In [ ]:
import arcpy

# Chemin vers shapefile d'entrée
input_layer = "C:/Users/liege/Desktop/TER/TEST_MODELE/Réseaux_l.shp" # ModelBuilder
# Chemin où sauvegarder les résultats exportés
output_folder = "C:/Users/liege/Desktop/TER/TEST_MODELE/COUCHE LINEAIRE SANS COTATION"

# Créer une couche temporaire valide pour la sélection
temp_layer = "temp_layer"  # Nom temporaire de la couche dans l'espace de travail

# Faire en sorte que la couche soit un objet de type 'Feature Layer'
arcpy.management.MakeFeatureLayer(input_layer, temp_layer)

# Obtenir la liste des types de réseaux présents dans la colonne 'type' du shapefile
types_reseau = set()  # Utilisation d'un set pour éviter les doublons
with arcpy.da.SearchCursor(temp_layer, ['type']) as cursor:
    for row in cursor:
        types_reseau.add(row[0])  # Ajouter le type de réseau au set

# Convertir le set en une liste ordonnée pour itérer dessus
types_reseau = sorted(types_reseau)

# Afficher les types trouvés (pour debug)
print("Types de réseaux trouvés :", types_reseau)

# Créer les shapefiles séparés pour chaque type de réseau
for type_reseau in types_reseau:
    # Créer la requête SQL pour filtrer les entités par type
    query = f"type = '{type_reseau}'"

    # Sélectionner les entités correspondant au type
    arcpy.management.SelectLayerByAttribute(temp_layer, "NEW_SELECTION", query)

    # Vérifier si des entités sont sélectionnées
    count = int(arcpy.management.GetCount(temp_layer)[0])
    if count > 0:
        # Définir le chemin de sortie pour chaque type de réseau
        output_fc = f"{output_folder}/{type_reseau}_Réseaux.shp"

        # Exporter les entités sélectionnées dans un shapefile spécifique
        arcpy.management.CopyFeatures(temp_layer, output_fc)

        print(f"Export du type {type_reseau} effectué avec succès : {output_fc}")
    else:
        print(f"Aucune entité trouvée pour le type {type_reseau}.")

# Libérer la mémoire utilisée pour la couche temporaire
arcpy.management.Delete(temp_layer)


**Jointure des champs de cotation (ponctuels) aux tronçons (linéaires)** (py3)Arcgis pro

Ce programme va créer un dossier qu’il remplira avec les shapefiles des 3 types de tronçons ayant des cotations (ici EP, EU et UNI) à partir de deux couches SIG créées par le ModelBuilder d’ArcGIS Pro.

Plusieurs couches temporaires seront créées par nécessité (tronçons et points) pour chaque type de réseau.

Le programme va prendre en considération le type de chaque entité pour ne mettre en relation que les cotations et les tronçons de même type. Il va également joindre les cotations les plus proches d’un tronçon (les cotations étant situées sur les tronçons, il n’y a donc pas d’erreur possible).

![image_py3](../images_colab/py3.png)

In [ ]:
import arcpy
import os
import re

# === PARAMÈTRES ===
points_path = r"C:\Users\liege\Desktop\TER\donnees general\ETIQUETTE_COTATION\COTATION_EP_EU_UNI.shp" # CSV du LISP n°1 converti en shapefiles
troncons_path = r"C:\Users\liege\Desktop\TER\TEST_MODELE\Réseaux_l.shp" # ModelBuilder
output_folder = r"C:\Users\liege\Desktop\TER\TEST_MODELE\COUCHE LINEAIRE AVEC COTATION"

# Créer le dossier de sortie si nécessaire
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Créer les couches temporaires
troncons_layer = "troncons_temp"
points_layer = "points_temp"
arcpy.management.MakeFeatureLayer(troncons_path, troncons_layer)
arcpy.management.MakeFeatureLayer(points_path, points_layer)

# Récupérer tous les types uniques dans les points
types_set = set()
with arcpy.da.SearchCursor(points_layer, ['type']) as cursor:
    for row in cursor:
        if row[0]:
            types_set.add(row[0])
types_list = sorted(types_set)
print("Types détectés :", types_list)

# Boucle principale
for reseau_type in types_list:
    print(f"\n--- Traitement du type : {reseau_type} ---")

    # Nettoyage du nom pour le fichier
    safe_type = re.sub(r'[^\w\-]', '_', reseau_type)

    # Création des couches filtrées
    troncons_sel = f"troncons_{safe_type}_lyr"
    points_sel = f"points_{safe_type}_lyr"
    clause = f"type = '{reseau_type}'"
    arcpy.management.MakeFeatureLayer(troncons_layer, troncons_sel, clause)
    arcpy.management.MakeFeatureLayer(points_layer, points_sel, clause)

    # === FieldMappings personnalisés ===
    field_mappings = arcpy.FieldMappings()

    # 1. Ajouter tous les champs des tronçons SAUF géométrie et champs système
    for field in arcpy.ListFields(troncons_sel):
        if field.type not in ['Geometry', 'OID'] and field.name.lower() not in ['shape_length', 'shape_area']:
            fmap = arcpy.FieldMap()
            fmap.addInputField(troncons_sel, field.name)
            field_mappings.addFieldMap(fmap)

    # 2. Ajouter uniquement 'id' et 'diametre' depuis les points (cotations)
    for field_name in ['id', 'diametre']:
        fmap = arcpy.FieldMap()
        fmap.addInputField(points_sel, field_name)
        output_field = fmap.outputField
        output_field.name = field_name
        output_field.aliasName = field_name
        fmap.outputField = output_field
        field_mappings.addFieldMap(fmap)

    # Sortie shapefile
    output_shp = os.path.join(output_folder, f"{safe_type}_Réseaux_Cotation.shp")

    # Jointure spatiale
    arcpy.analysis.SpatialJoin(
        target_features=troncons_sel,
        join_features=points_sel,
        out_feature_class=output_shp,
        join_type="KEEP_ALL",
        match_option="CLOSEST",
        field_mapping=field_mappings
    )

    print(f"✅ Jointure enregistrée : {output_shp}")

    # Nettoyage des couches temporaires
    arcpy.management.Delete(troncons_sel)
    arcpy.management.Delete(points_sel)

# Nettoyage final
arcpy.management.Delete(troncons_layer)
arcpy.management.Delete(points_layer)


**Séparation automatique des symboles (convertis en ponctuels) des réseaux en différents shapefiles selon leur type. (Sauf EP et EU)** (py4)Arcgis pro

Ce programme va lire l’ensemble du CSV et répartir les symboles (convertis en ponctuels) dans des shapefiles différents selon le type de réseau.

À partir d’une liste générée automatiquement grâce au champ « TYPE », il va ensuite créer des couches où il séparera individuellement chaque entité.

Le programme va utiliser le champ « TYPE » pour les répartir, ainsi que les champs « X » et « Y » pour générer leur géométrie dans le référentiel spatial du projet (CC48).

![image_py4_5](../images_colab/py4_5.png)

In [ ]:
import arcpy
import os
import pandas as pd
import codecs

# Chemins à modifier
csv_path = r"C:\Users\liege\Desktop\TER\donnees general\SYMBOLE DE RESEAU SAUF EP ET EU\Symboles_réseaux_sauf_EU_EP.csv" #CSV du LISP n°4
output_folder = r"C:\Users\liege\Desktop\TER\TEST_MODELE\SYMBOLE RESEAU SAUF EP ET EU"

# Création du dossier de sortie si inexistant
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lecture du CSV avec gestion de l'encodage latin1 (Windows-1252)
with codecs.open(csv_path, encoding='latin1', errors='replace') as f:
    df = pd.read_csv(f)

# Mise en majuscule et suppression des espaces sur les noms de colonnes
df.columns = [col.strip().upper() for col in df.columns]

# Vérifications des champs obligatoires
if "TYPE" not in df.columns:
    raise ValueError("Le champ 'TYPE' est obligatoire dans le CSV.")

if not {"X", "Y"}.issubset(df.columns):
    raise ValueError("Les champs 'X' et 'Y' sont obligatoires dans le CSV.")

# Liste des types uniques
types = df["TYPE"].dropna().unique()

# Référence spatiale CC48 = 3948
spatial_ref = arcpy.SpatialReference(3948)

for typ in types:
    typ_str = str(typ).strip()
    if not typ_str:
        continue

    df_typ = df[df["TYPE"] == typ]

    # Retirer X et Y des champs à garder dans le shapefile
    fields_to_keep = [col for col in df.columns if col not in ("X", "Y")]

    # Nom complet du shapefile de sortie
    out_shp = os.path.join(output_folder, f"Symboles_{typ_str}.shp")

    # Supprimer si déjà existant
    if arcpy.Exists(out_shp):
        arcpy.Delete_management(out_shp)

    # Création du shapefile point
    arcpy.management.CreateFeatureclass(output_folder, f"Symboles_{typ_str}.shp", "POINT", spatial_reference=spatial_ref)

    # Liste des champs existants
    existing_fields = [f.name for f in arcpy.ListFields(out_shp)]

    # Ajout des champs attributaires
    for field in fields_to_keep:
        field_name = field[:10]  # Nom limité à 10 caractères (limite shapefile)
        if field_name not in existing_fields:
            arcpy.AddField_management(out_shp, field_name, "TEXT", field_length=254)

    # Insertion des entités avec attributs
    with arcpy.da.InsertCursor(out_shp, ["SHAPE@XY"] + fields_to_keep) as cursor:
        for idx, row in df_typ.iterrows():
            xy = (row["X"], row["Y"])
            attrs = [str(row[f]) if pd.notna(row[f]) else None for f in fields_to_keep]
            cursor.insertRow([xy] + attrs)

    print(f"Shapefile créé pour TYPE={typ_str} : {out_shp}")

print("Tous les shapefiles ont été générés.")


**Séparation automatique des points des réseaux en différents shapefiles selon leur type (sauf EP et EU)** (py5) Arcgis pro

Ce programme va lire l’ensemble du CSV et répartir les points dans des shapefiles différents selon le type de réseau.

*(Voir py4 car très ressemblant)*


![image_py4_5](../images_colab/py4_5.png)

In [ ]:
import arcpy
import os
import csv

# Paramètres
csv_path = r"C:\Users\liege\Desktop\TER\donnees general\COTATION POINTS\cotations_points_export.csv" #CSV du LISP n°2
output_folder = r"C:\Users\liege\Desktop\TER\TEST_MODELE\COTATION POINTS"
gdb_name = "CotationPoints.gdb"
gdb_path = os.path.join(output_folder, gdb_name)
sr = arcpy.SpatialReference(3948)  # CC48

# Création du dossier de sortie s'il n'existe pas
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Création de la FileGDB si elle n'existe pas déjà
if not arcpy.Exists(gdb_path):
    print(f"Création de la géodatabase : {gdb_path}")
    arcpy.management.CreateFileGDB(output_folder, gdb_name)
else:
    print(f"Géodatabase existante : {gdb_path}")

# Fonction pour créer une feature class avec les champs requis
def create_feature_class(path, spatial_ref):
    print(f"Création de la feature class : {path}")
    arcpy.management.CreateFeatureclass(os.path.dirname(path), os.path.basename(path), "POINT", spatial_reference=spatial_ref)
    arcpy.management.AddField(path, "Nom", "TEXT", field_length=50)
    arcpy.management.AddField(path, "TYPE", "TEXT", field_length=10)
    arcpy.management.AddField(path, "Zreseau", "DOUBLE", nullable="NULLABLE")  # accepte NULL
    arcpy.management.AddField(path, "PROF", "DOUBLE", nullable="NULLABLE")     # accepte NULL
    arcpy.management.AddField(path, "Layer", "TEXT", field_length=100)

# Lire le CSV et grouper par TYPE
points_by_type = {}

with open(csv_path, encoding='latin1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        typ = row["TYPE"].strip()
        if typ == "":
            continue
        if typ not in points_by_type:
            points_by_type[typ] = []
        points_by_type[typ].append(row)

# Pour chaque type, créer une feature class dans la GDB et insérer les points
for typ, rows in points_by_type.items():
    fc_name = f"Points_{typ}"
    fc_path = os.path.join(gdb_path, fc_name)

    if not arcpy.Exists(fc_path):
        create_feature_class(fc_path, sr)

    with arcpy.da.InsertCursor(fc_path, ["SHAPE@XY", "Nom", "TYPE", "Zreseau", "PROF", "Layer"]) as cursor:
        for row in rows:
            try:
                x = float(row["X"])
                y = float(row["Y"])
                # None pour valeurs vides
                zreseau = float(row["Zreseau"]) if row["Zreseau"] != "" else None
                prof = float(row["PROF"]) if row["PROF"] != "" else None
            except ValueError:
                continue
            cursor.insertRow(((x, y), row["Nom"], typ, zreseau, prof, row["Layer"]))

print("Export terminé. Toutes les feature classes sont dans la géodatabase :", gdb_path)


**Transformation de l’étiquette AutoCAD en champs pour le SIG** (py6) COLAB

Ce programme va lire l’ensemble du CSV et distribuer les informations du champ texte en un champ ID, Z tampon et Z radier, puis va créer un champ qui mettra au propre l’ancienne étiquette AutoCAD afin qu’elle puisse être présentée sur ArcGIS Pro.

![image_py6](../images_colab/py6.png)

In [ ]:
import csv
import re

# ---------- Extraction des valeurs principales ----------
def extraire_donnees(texte):
    id_match = re.search(r'ID\{.*?Regard\}:\s*(REG_[A-Z]+[0-9]+)', texte)
    tampon_match = re.search(r'Tampon\}:\s*([0-9]+\.[0-9]+)', texte)
    radier_match = re.search(r'Radier\}:\s*([0-9]+\.[0-9]+)', texte)

    id_regard = id_match.group(1) if id_match else ''
    z_tampon = tampon_match.group(1) if tampon_match else ''
    z_radier = radier_match.group(1) if radier_match else ''

    return id_regard, z_tampon, z_radier

# ---------- Extraction des Zfe ----------
def extraire_zfe(texte):
    zfe_matches = re.findall(r'Zfe\{.*?\((.*?)\)\}:\s*([0-9]+\.[0-9]+)(?:\s*\{\\H[^;]+;\((.*?)\)\})?', texte)
    return zfe_matches

# ---------- Construction de l'étiquette ----------
def convertir_pour_arcgis(texte):
    lignes = []

    id_regard, z_tampon, z_radier = extraire_donnees(texte)
    if id_regard:
        lignes.append(f"IDRegard: {id_regard}")
    if z_tampon:
        lignes.append(f"ZTampon: {z_tampon}m")
    if z_radier:
        lignes.append(f"ZRadier: {z_radier}m")

    zfe_list = extraire_zfe(texte)
    for nom, valeur, annotation in zfe_list:
        ligne = f"Zfe ({nom}): {valeur}m"
        if annotation:
            ligne += f" ({annotation})"
        lignes.append(ligne)

    return '\n'.join(lignes)

# ---------- Traitement du fichier ----------
def traiter_fichier(input_csv, output_csv):
    with open(input_csv, mode='r', encoding='latin1', newline='') as infile, \
         open(output_csv, mode='w', encoding='latin1', newline='') as outfile:

        reader = csv.DictReader(infile, delimiter=';', quotechar='"')
        # Ajout des nouveaux champs en fin de liste sans duplication
        extra_fields = ['IDRegard', 'ZTampon', 'ZRadier', 'EtiArcGIS']
        fieldnames = reader.fieldnames + [f for f in extra_fields if f not in reader.fieldnames]

        writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writeheader()

        for row in reader:
            texte = row.get('Texte', '')
            id_regard, z_tampon, z_radier = extraire_donnees(texte)
            etiquette = convertir_pour_arcgis(texte)

            row['IDRegard'] = id_regard
            row['ZTampon'] = z_tampon
            row['ZRadier'] = z_radier
            row['EtiArcGIS'] = etiquette

            writer.writerow(row)

# ---------- Chemins des fichiers ----------
input_path = '/content/Etiquette_EU_EP.csv'  #CSV du LISP n°5
output_path = '/content/Etiquette_EU_EP_presentation.csv'

# ---------- Lancement du traitement ----------
print(f"Traitement du fichier : {input_path}")
traiter_fichier(input_path, output_path)
print(f"Fichier converti généré : {output_path}")


**Jointure des étiquettes des symboles de réseau EU et EP avec leur position de bloc** (py7) Arcgis pro

Ce programme lit les deux fichiers CSV de symboles (EU et EP) et effectue une jointure spatiale avec le fichier CSV des étiquettes. Il associe à chaque symbole les champs suivants provenant des étiquettes : IDRegard, ZTampon, ZRadier et EtiArcGIS.

![image_py7](../images_colab/py7.png)

In [ ]:
import arcpy
import os
import shutil
import time

arcpy.env.overwriteOutput = True

# Chemins d'entrée (CSV et dossier de sortie)
csv_symboles_ep = r"C:\Users\liege\Downloads\donnees_general\Symboles_reseau_EP.csv"
csv_symboles_eu = r"C:\Users\liege\Downloads\donnees_general\Symboles_reseau_EU.csv"
csv_etiquettes = r"C:\Users\liege\Downloads\donnees_general\Etiquette_EU_EP_presentation.csv"

output_folder = r"C:\Users\liege\Desktop\TER\TEST_MODELE\SYMBOLE RESEAU EP ET EU"
temp_workspace = r"C:\Users\liege\Downloads\donnees_general\temp_arcpy"

if not os.path.exists(temp_workspace):
    os.makedirs(temp_workspace)

# Système de coordonnées
sr = arcpy.SpatialReference(3948)

def csv_to_shapefile(input_csv, out_shp, x_field="X", y_field="Y", z_field=None):
    arcpy.management.XYTableToPoint(
        in_table=input_csv,
        out_feature_class=out_shp,
        x_field=x_field,
        y_field=y_field,
        z_field=z_field,
        coordinate_system=sr
    )
    print(f"Shapefile créé : {out_shp}")

def get_field_names(layer):
    return [f.name for f in arcpy.ListFields(layer) if f.type not in ("OID", "Geometry")]

def jointure_spatiale(symboles_shp, etiquettes_shp, out_shp, type_val):
    field_mappings = arcpy.FieldMappings()

    # Ajouter tous les champs de la couche symboles
    for champ in get_field_names(symboles_shp):
        try:
            fm = arcpy.FieldMap()
            fm.addInputField(symboles_shp, champ)
            fm.outputField.name = champ
            fm.outputField.aliasName = champ
            field_mappings.addFieldMap(fm)
        except Exception as e:
            print(f"Erreur ajout champ symbole '{champ}': {e}")

    # Champs à conserver depuis les étiquettes
    champs_etiquettes_a_conserver = ["EtiArcGIS", "IDRegard", "ZTampon", "ZRadier"]
    champs_disponibles = get_field_names(etiquettes_shp)

    for champ in champs_etiquettes_a_conserver:
        if champ in champs_disponibles:
            try:
                fm = arcpy.FieldMap()
                fm.addInputField(etiquettes_shp, champ)
                fm.outputField.name = champ
                fm.outputField.aliasName = champ
                field_mappings.addFieldMap(fm)
            except Exception as e:
                print(f"Erreur ajout champ etiquette '{champ}': {e}")
        else:
            print(f"Champ {champ} absent dans les données d’étiquettes.")

    where_clause = f"Type = '{type_val}'"
    etiquettes_filtrees = os.path.join(temp_workspace, f"etiquettes_filtrees_{type_val}.shp")

    if arcpy.Exists(etiquettes_filtrees):
        arcpy.Delete_management(etiquettes_filtrees)

    arcpy.management.MakeFeatureLayer(etiquettes_shp, "etiquettes_lyr")
    arcpy.management.SelectLayerByAttribute("etiquettes_lyr", "NEW_SELECTION", where_clause)
    arcpy.management.CopyFeatures("etiquettes_lyr", etiquettes_filtrees)

    join_result = os.path.join(temp_workspace, f"join_result_{type_val}.shp")
    if arcpy.Exists(join_result):
        arcpy.Delete_management(join_result)

    arcpy.analysis.SpatialJoin(
        target_features=symboles_shp,
        join_features=etiquettes_filtrees,
        out_feature_class=join_result,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping=field_mappings,
        match_option="CLOSEST",
        search_radius="5 Meters"
    )

    if arcpy.Exists(out_shp):
        arcpy.Delete_management(out_shp)
    arcpy.management.CopyFeatures(join_result, out_shp)
    print(f"Jointure spatiale terminée, fichier : {out_shp}")

def clean_temp(temp_dir):
    try:
        time.sleep(1)
        arcpy.ClearWorkspaceCache_management()
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
            print(f"Dossier temporaire supprimé : {temp_dir}")
    except Exception as e:
        print(f"Erreur lors de la suppression du dossier temporaire : {e}")

def main():
    shp_symboles_ep = os.path.join(output_folder, "Symboles_EP.shp")
    shp_symboles_eu = os.path.join(output_folder, "Symboles_EU.shp")
    csv_to_shapefile(csv_symboles_ep, shp_symboles_ep)
    csv_to_shapefile(csv_symboles_eu, shp_symboles_eu)

    shp_etiquettes = os.path.join(output_folder, "Etiquettes_EP_EU.shp")
    csv_to_shapefile(csv_etiquettes, shp_etiquettes)

    out_jointure_ep = os.path.join(output_folder, "Symboles_EP_EtiArcGIS.shp")
    jointure_spatiale(shp_symboles_ep, shp_etiquettes, out_jointure_ep, type_val="EP")

    out_jointure_eu = os.path.join(output_folder, "Symboles_EU_EtiArcGIS.shp")
    jointure_spatiale(shp_symboles_eu, shp_etiquettes, out_jointure_eu, type_val="EU")

    clean_temp(temp_workspace)

if __name__ == "__main__":
    main()
